In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, f1_score

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
main_dir = 'drive/MyDrive/23Spring-MLMA/isolated/'
os.listdir(main_dir)

['ReadMe',
 '002',
 '005',
 '004',
 '003',
 '001',
 '006',
 'XGboost0.pkl',
 'XGboost1.pkl',
 'XGboost2.pkl',
 'XGboost3.pkl',
 'XGboost4.pkl',
 'XGboost5.pkl',
 'XGboost_patient_0model_1.pkl',
 'XGboost_patient_0model_3.pkl',
 'XGboost_patient_0model_5.pkl',
 'XGboost_patient_0model_7.pkl',
 'XGboost_patient_0model_9.pkl',
 'XGboost_patient_1model_1.pkl',
 'XGboost_patient_1model_3.pkl',
 'XGboost_patient_1model_5.pkl',
 'XGboost_patient_1model_7.pkl',
 'XGboost_patient_1model_9.pkl',
 'XGboost_patient_2model_1.pkl',
 'XGboost_patient_2model_3.pkl',
 'XGboost_patient_2model_5.pkl',
 'XGboost_patient_2model_7.pkl',
 'XGboost_patient_2model_9.pkl',
 'XGboost_patient_3model_1.pkl',
 'XGboost_patient_3model_3.pkl',
 'XGboost_patient_3model_5.pkl',
 'XGboost_patient_3model_7.pkl',
 'XGboost_patient_3model_9.pkl',
 'XGboost_patient_4model_1.pkl',
 'XGboost_patient_4model_3.pkl',
 'XGboost_patient_4model_5.pkl',
 'XGboost_patient_4model_7.pkl',
 'XGboost_patient_4model_9.pkl',
 'XGboost_pati

In [50]:
# get data
def get_original_feature_label(path):
    # scale both vertical and horizontal between 0 and 1
    scaler = MinMaxScaler()
    feature = []
    label = []
    for i in os.listdir(path):
        a = pd.read_csv(str(path+i),names=["vertical","horizontal"])
        a = np.array(a)
        a = scaler.fit_transform(a)
        a = np.swapaxes(a,0,1)
        feature.append(a)
        if i.endswith("001.csv"):
            label.append(0)
        elif i.endswith("002.csv"):
            label.append(1)
        elif i.endswith("003.csv"):
            label.append(2)
        elif i.endswith("004.csv"):
            label.append(3)
        elif i.endswith("005.csv"):
            label.append(4)
        elif i.endswith("006.csv"):
            label.append(5)
        elif i.endswith("007.csv"):
            label.append(6)
        elif i.endswith("008.csv"):
            label.append(7)
        elif i.endswith("009.csv"):
            label.append(8)
        elif i.endswith("010.csv"):
            label.append(9)
        elif i.endswith("011.csv"):
            label.append(10)
        elif i.endswith("012.csv"):
            label.append(11)
    
    return feature,label

# Split on patient level - User-independent


> 6-fold cross-validation





In [13]:
import xgboost as xgb
def train(train_feature,train_label,test_feature,test_label,num):
    # 定义模型
    model = xgb.XGBClassifier(objective='multi:softmax', num_class=12, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)

    # 预测测试集
    y_pred = model.predict(test_feature)

    # 计算准确率
    accuracy = (y_pred == test_label).sum() / len(test_label)
    # auc = roc_auc_score(test_label, y_pred, multi_class='ovr')
    average_method = 'macro' # 'macro' 或 'micro' 或 'weighted'
    f1 = f1_score(test_label, y_pred,average=average_method)#, multi_class='ovr')
    print('Accuracy:', accuracy)
    # print('AUC:', auc)
    print('F1:', f1)

    # 保存模型
    pickle.dump(model, open(main_dir+"XGboost"+str(num)+".pkl", 'wb'))

    return accuracy,f1#auc,f1

In [51]:
import xgboost as xgb
def train(train_feature,train_label,test_feature,test_label,num):
    # 定义模型
    model = xgb.XGBClassifier(objective='multi:softprob', eval_metric = 'auc',num_class=12, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)#,eval_set=[(test_feature, test_label)])

    # 预测测试集
    y_pred = model.predict_proba(test_feature)
    # print(np.shape(y_pred))
    # print(y_pred)

    # 计算准确率
    # accuracy = (y_pred == test_label).sum() / len(test_label)
    auc = roc_auc_score(test_label, y_pred, multi_class='ovr')
    # average_method = 'macro' # 'macro' 或 'micro' 或 'weighted'
    # f1 = f1_score(test_label, y_pred,average=average_method)#, multi_class='ovr')
    # print('Accuracy:', accuracy)
    print('AUC:', auc)
    # print('F1:', f1)

    # 保存模型
    pickle.dump(model, open(main_dir+"XGboost"+str(num)+".pkl", 'wb'))

    return auc#accuracy,f1#auc,f1

In [ ]:
import xgboost as xgb
def train(train_feature,train_label,test_feature,test_label,num):
    # 定义模型
    model = xgb.XGBClassifier(objective='multi:softprob', eval_metric = 'auc',num_class=12, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)#,eval_set=[(test_feature, test_label)])

    # 预测测试集
    y_pred_p = model.predict_proba(test_feature)
    y_pred = model.predictor(test_feature)

    # 计算准确率 auc f1
    accuracy = (y_pred == test_label).sum() / len(test_label)
    auc = roc_auc_score(test_label, y_pred_p, multi_class='ovr')
    average_method = 'macro' # 'macro' 或 'micro' 或 'weighted'
    f1 = f1_score(test_label, y_pred,average=average_method)#, multi_class='ovr')
    print('Accuracy:', accuracy)
    print('AUC:', auc)
    print('F1:', f1)

    # 保存模型
    pickle.dump(model, open(main_dir+"XGboost"+str(num)+".pkl", 'wb'))

    return accuracy, auc, f1

In [52]:
patient_all = ["001",'002','003','004','005','006']
accl = [];aucl = []; f1l = []
for j in range(6):
  patient = [patient_all[j]]
  test_feature = []
  test_label = []
  for i in patient:
      f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
      for x in f:
          test_feature.append(x)         
      for x in l:
          test_label.append(x)
  print(patient)
  print(len(test_feature))
  test_feature = np.reshape(test_feature,(np.shape(test_feature)[0],-1))

  patient = [x for x in patient_all if x != patient_all[j]]
  print(patient)
  train_feature = []
  train_label = []
  for i in patient:
      f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
      for x in f:
          train_feature.append(x)         
      for x in l:
          train_label.append(x)
  print(len(train_feature))
  train_feature = np.reshape(train_feature,(np.shape(train_feature)[0],-1))
  acc, auc, f1 = train(train_feature,train_label,test_feature,test_label,j)
  accl.append(acc)
  aucl.append(auc)
  f1l.append(f1)
  # print(acc)

['001']
120
['002', '003', '004', '005', '006']
604
AUC: 0.9242424242424242
['002']
120
['001', '003', '004', '005', '006']
604
AUC: 0.9872727272727273
['003']
122
['001', '002', '004', '005', '006']
602
AUC: 0.8887490981240981
['004']
121
['001', '002', '003', '005', '006']
603
AUC: 0.9090927704564068
['005']
120
['001', '002', '003', '004', '006']
604
AUC: 0.9597680979561712
['006']
121
['001', '002', '003', '004', '005']
603
AUC: 0.9712648599012236


In [53]:
np.mean(aucl)

0.9400649963255087

In [15]:
print(np.mean(accl))

0.6577718315795812


In [16]:
print(np.mean(aucl))

nan


/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
print(np.mean(f1l))

0.6360594070761618


# User-dependent


> 5-fold cross-validation





In [28]:
# get data
def get_original_feature_label(path, t1, t2):
    # scale both vertical and horizontal between 0 and 1
    scaler = MinMaxScaler()
    feature = []
    label = []
    if t2==0:
        num = len(os.listdir(path))-2
    else:
        num = 2
    for i in os.listdir(path):
        a = pd.read_csv(str(path+i),names=["vertical","horizontal"])
        a = np.array(a)
        a = scaler.fit_transform(a)
        a = np.swapaxes(a,0,1)
        if (t1==1)&(t2!=0)&((i[8:10]=="01") or (i[8:10]=="02")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==3)&(t2!=0)&((i[8:10]=="03") or (i[8:10]=="04")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==5)&(t2!=0)&((i[8:10]=="05") or (i[8:10]=="06")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==7)&(t2!=0)&((i[8:10]=="07") or (i[8:10]=="08")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==9)&(t2!=0)&((i[8:10]=="09") or (i[8:10]=="10")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue

        if (t1==1)&(t2==0)&((i[8:10]=="01") or (i[8:10]=="02")):
           continue
        if (t1==3)&(t2==0)&((i[8:10]=="03") or (i[8:10]=="04")):
           continue
        if (t1==5)&(t2==0)&((i[8:10]=="05") or (i[8:10]=="06")):
           continue
        if (t1==7)&(t2==0)&((i[8:10]=="07") or (i[8:10]=="08")):
           continue
        if (t1==9)&(t2==0)&((i[8:10]=="09") or (i[8:10]=="10")):
           continue

        if (t2==0):
          feature.append(a)
          if i.endswith("001.csv"):
              label.append(0)
          elif i.endswith("002.csv"):
              label.append(1)
          elif i.endswith("003.csv"):
              label.append(2)
          elif i.endswith("004.csv"):
              label.append(3)
          elif i.endswith("005.csv"):
              label.append(4)
          elif i.endswith("006.csv"):
              label.append(5)
          elif i.endswith("007.csv"):
              label.append(6)
          elif i.endswith("008.csv"):
              label.append(7)
          elif i.endswith("009.csv"):
              label.append(8)
          elif i.endswith("010.csv"):
              label.append(9)
          elif i.endswith("011.csv"):
              label.append(10)
          elif i.endswith("012.csv"):
              label.append(11)
        
    
    return feature,label

In [ ]:
import xgboost as xgb
def train(train_feature,train_label,test_feature,test_label,num,p):
    # 定义模型
    model = xgb.XGBClassifier(objective='multi:softprob', eval_metric = 'auc',num_class=12, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)#,eval_set=[(test_feature, test_label)])

    # 预测测试集
    y_pred_p = model.predict_proba(test_feature)
    y_pred = model.predict(test_feature)

    # 计算准确率 auc f1
    accuracy = (y_pred == test_label).sum() / len(test_label)
    auc = roc_auc_score(test_label, y_pred_p, multi_class='ovr')
    average_method = 'macro' # 'macro' 或 'micro' 或 'weighted'
    f1 = f1_score(test_label, y_pred,average=average_method)#, multi_class='ovr')
    print('Accuracy:', accuracy)
    print('AUC:', auc)
    print('F1:', f1)

    # 保存模型
    pickle.dump(model, open(main_dir+"XGboost_patient_"+str(p)+"model_"+str(num)+".pkl", 'wb'))

    return accuracy,auc,f1

In [46]:
patient_all = ["001",'002','003','004','005','006']
acc_p = []; auc_p = []; f1_p = []
for j in range(6):

  patient_model = patient_all[j]
  patient_path = str(main_dir+patient_model+"/isolated_strokes/")
  accl = []; aucl = []; f1l = []

  for i in range(1,10,2):
    train_feature = []
    train_label = []
    f,l = get_original_feature_label(patient_path,i,0)
    for x in f:
        train_feature.append(x)         
    for x in l:
        train_label.append(x)
    print(len(train_label))

    test_feature = []
    test_label = []
    f,l = get_original_feature_label(patient_path,i,1)
    for x in f:
        test_feature.append(x)         
    for x in l:
        test_label.append(x)
    print(len(test_label))
    
    test_feature = np.reshape(test_feature,(np.shape(test_feature)[0],-1))
    train_feature = np.reshape(train_feature,(np.shape(train_feature)[0],-1))
    acc, auc, f1 = train(train_feature,train_label,test_feature,test_label,i,j)
    accl.append(acc)
    aucl.append(auc)
    f1l.append(f1)
    
  acc_p.append(np.mean(accl))
  auc_p.append(np.mean(aucl))
  f1_p.append(np.mean(f1l))

96
24
AUC: 0.9015151515151514
96
24
AUC: 0.9734848484848485
96
24
AUC: 0.9166666666666666
96
24
AUC: 1.0
96
24
AUC: 0.9829545454545454
96
24
AUC: 0.9886363636363638
96
24
AUC: 0.9242424242424242
96
24
AUC: 0.9659090909090908
96
24
AUC: 0.9867424242424243
96
24
AUC: 0.9640151515151515
98
24
AUC: 0.9109848484848485
98
24
AUC: 0.9166666666666665
98
24
AUC: 0.9810606060606061
98
24
AUC: 0.962121212121212
98
24
AUC: 0.9772727272727272
97
24
AUC: 0.8579545454545454
97
24
AUC: 0.9450757575757577
97
24
AUC: 0.9488636363636364
97
24
AUC: 0.8598484848484848
97
24
AUC: 0.9034090909090909
96
24
AUC: 0.9337121212121211
96
24
AUC: 0.9583333333333334
96
24
AUC: 0.9943181818181818
96
24
AUC: 0.9583333333333334
97
23
AUC: 0.9193722943722943
97
24
AUC: 0.9356060606060607
97
24
AUC: 0.9962121212121212
97
24
AUC: 0.9753787878787877
97
24
AUC: 0.9867424242424242
97
24
AUC: 0.9602272727272726


In [47]:
np.mean(auc_p)

0.9495220057720059

In [26]:
np.mean(acc_p)

0.7342995169082126

In [27]:
np.mean(f1_p)

0.7101190476190476